In [109]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict

## Configuration 

In [110]:
hi_df = pd.DataFrame()
hi_df = pd.read_csv( 'file/sample_output/healthinformation.csv') 
#data_df = data_df.to_dict(orient='records')
hi_df

,_id,_index_map,formid,answers.height,answers.bloodtype,answers.identification.number.code,answers.identification.type,answers.identification.number.pin,answers.income,answers.bp1_systolic
0,aQMjmsDcwMbPs5NyhqQPUZ,NaN,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120
1,DexsDsw39ss5NyhqQPUP,NaN,DexsDsw39ss5NyhqQPUP,23,B,2033.2,SSS,333,80.0,201


In [111]:
def set_for_keys(my_dict, key_arr, val):

    current = my_dict
    for i in range(len(key_arr)):
        key = key_arr[i]
        if key not in current:
            if i==len(key_arr)-1:
                current[key] = val
            else:
                current[key] = {}
        else:
            if type(current[key]) is not dict:
                print("not compatible with key structure requested")
                raise ValueError("Dictionary key already occupied")

        current = current[key]

    return my_dict

def to_formatted_json(df, sep="."):
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            
            parsed_row = set_for_keys(parsed_row, keys, val)
        result.append(parsed_row)

    
    return result

def to_formatted_json_test(df, sep="."):
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            
            parsed_row = set_for_keys(parsed_row, keys, val)
            
            print('keys: ' , keys)
            print('values: ' , val)
            print('idx: ' , idx)
            print('----------')
            #print('parse_row: ',parsed_row)
        print(parsed_row)
        print('-----parsed_row-----')
        result.append(parsed_row)
        
    
    return result

#Where df was parsed from json-dict using json_normalize

In [112]:
#hi_df = to_formatted_json(data_df, sep=".")
#hi_df

## File loader

In [113]:
fdh_df  = pd.DataFrame()
fdh_df  = pd.read_csv( 'file/sample_output/answers.familydiseasehistory.csv') 
#data_df = data_df.to_dict(orient='records')
fdh_df = fdh_df.sort_values(['_id','_index_map'])
fdh_df

,_id,_index_map,answers.familydiseasehistory
3,DexsDsw39ss5NyhqQPUP,0,Head Cancer
4,DexsDsw39ss5NyhqQPUP,1,Stomach Cancer
0,aQMjmsDcwMbPs5NyhqQPUZ,0,"Respiratory ,disease"
1,aQMjmsDcwMbPs5NyhqQPUZ,1,Brain Cancer
2,aQMjmsDcwMbPs5NyhqQPUZ,2,Stomach Cancer


## GROUP BY IF FILE IS ALL ARRAY

In [114]:
fdh_df = fdh_df.groupby('_id').agg(lambda x: x.tolist())
fdh_df = fdh_df.reset_index()
fdh_df

,_id,_index_map,answers.familydiseasehistory
0,DexsDsw39ss5NyhqQPUP,"[0, 1]","[Head Cancer, Stomach Cancer]"
1,aQMjmsDcwMbPs5NyhqQPUZ,"[0, 1, 2]","[Respiratory ,disease, Brain Cancer, Stomach C..."


In [115]:
fdh_df = fdh_df.drop(['_index_map'], axis=1)
fdh_df

,_id,answers.familydiseasehistory
0,DexsDsw39ss5NyhqQPUP,"[Head Cancer, Stomach Cancer]"
1,aQMjmsDcwMbPs5NyhqQPUZ,"[Respiratory ,disease, Brain Cancer, Stomach C..."


In [116]:
merge_join = pd.merge(hi_df,fdh_df, left_on='_id', right_on='_id', how='left')
merge_join = merge_join.drop(['_id'], axis=1).drop(['_index_map'], axis=1).replace(np.nan,'')
merge_join


,formid,answers.height,answers.bloodtype,answers.identification.number.code,answers.identification.type,answers.identification.number.pin,answers.income,answers.bp1_systolic,answers.familydiseasehistory
0,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120,"[Respiratory ,disease, Brain Cancer, Stomach C..."
1,DexsDsw39ss5NyhqQPUP,23,B,2033.2,SSS,333,80.0,201,"[Head Cancer, Stomach Cancer]"


## JOIN SHOULD BE IN THE PERSISTENT DATABASE ALREADY

In [ ]:
add_df  = pd.DataFrame()
add_df  = pd.read_csv( 'file/sample_output/answers.contactnumber.csv') 
#add_df = add_df.sort_values(['_id','_index_map'])
add_df

In [ ]:
add_street_df  = pd.DataFrame()
add_street_df = pd.read_csv( 'file/sample_output/answers.contactnumber.ext.csv') 
add_street_df = add_street_df.sort_values(['_id','_index_map'])
#add_street_df = add_street_df.drop(['_id'], axis=1).drop(['_index_map'], axis=1).replace(np.nan,'')
#add_street_df = add_street_df.drop(['_index_map'], axis=1).replace(np.nan,'')
add_street_df 

## Convert object into array of object

In [139]:
format_json = to_formatted_json(add_street_df, sep=".")
format_json

[{'_id': 'DexsDsw39ss5NyhqQPUP',
  '_index_map': 0.0,
  'answers': {'contactnumber': {'ext': {'number': 9,
     'trunkline': 'Reception'}}}},
 {'_id': 'DexsDsw39ss5NyhqQPUP',
  '_index_map': 0.1,
  'answers': {'contactnumber': {'ext': {'number': 5,
     'trunkline': 'Operator'}}}},
 {'_id': 'DexsDsw39ss5NyhqQPUP',
  '_index_map': 1.0,
  'answers': {'contactnumber': {'ext': {'number': 23,
     'trunkline': 'Mobile Trunkline'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 0.0,
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Reception'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 1.0,
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Mobile Trunkline'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 1.1,
  'answers': {'contactnumber': {'ext': {'number': 3,
     'trunkline': 'Operator'}}}}]

In [125]:
result = [
    {
        "standard": "119",
        "score": "0",
        "type": "assignment",
        "student": "4"
    },
    {
        "standard": "119",
        "score": "0",
        "type": "assignment",
        "student": "5"
    },
    {
        "standard": "118",
        "score": "0",
        "type": "assessment",
        "student": "4"
    }
]

In [126]:
def conf_to_nested_dict(keys, result):
    R = {}
    for record in result: 
        node = R
        for key in keys[:-1]:
            kv = record[key]
            next_node = node.get(kv, {})
            node[kv] = next_node
            node = next_node
        last_node = node.get(record[keys[-1]], [])
        last_node.append(record)
        node[record[keys[-1]]] = last_node


    return R

In [166]:
shit = conf_to_nested_dict(['standard','type'],result)
shit

{'119': {'assignment': [{'standard': '119',
    'score': '0',
    'type': 'assignment',
    'student': '4'},
   {'standard': '119', 'score': '0', 'type': 'assignment', 'student': '5'}]},
 '118': {'assessment': [{'standard': '118',
    'score': '0',
    'type': 'assessment',
    'student': '4'}]}}

In [163]:
add_street_df  = pd.DataFrame()
add_street_df = pd.read_csv( 'file/sample_output/answers.contactnumber.ext.csv') 
add_street_df = add_street_df.sort_values(['_id','_index_map']).reset_index(drop=True)
add_street_df

,_id,_index_map,answers.contactnumber.ext.number,answers.contactnumber.ext.trunkline
0,DexsDsw39ss5NyhqQPUP,0.0,9,Reception
1,DexsDsw39ss5NyhqQPUP,0.1,5,Operator
2,DexsDsw39ss5NyhqQPUP,1.0,23,Mobile Trunkline
3,aQMjmsDcwMbPs5NyhqQPUZ,0.0,2,Reception
4,aQMjmsDcwMbPs5NyhqQPUZ,1.0,2,Mobile Trunkline
5,aQMjmsDcwMbPs5NyhqQPUZ,1.1,3,Operator


In [379]:
d = {}
w = {}

w['number'] = 1
w['trunkline'] = 'operation'

d.setdefault('ext', [None]*2)
d['ext'][0] = {}
d['ext'][1] = {}
#d['ext'][0] = {}
#d['ext'][0].update(w)
#d['ext'][0].update(y)
#d['ext'][1] = {}
#d['ext'][1].update(z)

d

{'ext': [{}, {}]}

In [300]:
a = {}
a.setdefault('contactnumber', [None]*2)
a['contactnumber'][0] = {}
a['contactnumber'][0].update(d)

a

{'contactnumber': [{'ext': [{'Number': '02', 'Trunkline': 'Operation'},
    {'Number': '32', 'Trunkline': 'Reception'}]},
  None]}

In [348]:
src = "answers.contactnumber/0.ext/0.number"

In [349]:
for item in src.split('.'):
    print(item)



answers
contactnumber/0
ext/0
number
